# <span style="color:green"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Canalización  de datos. La API tf.data</center></span>

## <span style="color:blue">Escritor</span>

1. Oleg Jarma, ojarmam@unal.edu.co 

##   <span style="color:blue">Profesores</span>

2. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
3. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
4. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

5. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

6. Laura Lizarazo, ljlizarazore@unal.edu.co 
7. Julieth Lopez, julalopezcas@unal.edu.co

## <span style="color:blue">Contenido</span> 

## <span style="color:blue">Introducción</span> 

Basado en [tf.data](https://www.tensorflow.org/guide/data).

La API `tf.data` permite crear tuberías de entrada complejas a partir de piezas simples y reutilizables. Por ejemplo, la canalización de un modelo de imagen podría agregar datos de archivos en un sistema de archivos distribuido, aplicar perturbaciones aleatorias a cada imagen y fusionar imágenes seleccionadas al azar en un lote para entrenamiento. La canalización de un modelo de texto puede implicar extraer símbolos de datos de texto sin procesar, convertirlos en identificadores incrustados con una tabla de búsqueda y agrupar secuencias de diferentes longitudes. 

## <span style="color:blue">Importa librerías</span> 

In [1]:
import tensorflow as tf

import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

2021-10-22 20:24:54.231435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## <span style="color:blue">Esenciales</span> 

Para crear una canalización de entrada, debe comenzar con una fuente de datos. Por ejemplo, para construir un `Dataset` de datos a partir de datos en la memoria, puede usar *tf.data.Dataset.from_tensors()* o *tf.data.Dataset.from_tensor_slices()*. Alternativamente, si sus datos de entrada están almacenados en un archivo en el formato *TFRecord* de TensorFlow puede usar *tf.data.TFRecordDataset()*.

El objeto Dataset es un iterable de Python. Esto hace posible consumir sus elementos usando un bucle for:

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
dataset

2021-10-22 20:24:55.367167: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-22 20:24:55.367861: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-22 20:24:55.385686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 20:24:55.386574: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2021-10-22 20:24:55.386599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-22 20:24:55.470300: I tensorflow/stream_executor/platform/defa

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
len(dataset)

6

In [4]:
for elem in dataset:
    print(elem.numpy())

8
3
0
8
2
1


o se pueden crear explícitamente un iterador

In [5]:
it = iter(dataset)
print(next(it).numpy())
print(next(it).numpy())

8
3


### Consumo de datos usando reducción: reduce

In [6]:
print(dataset.reduce(0, lambda state, value: state+value).numpy())

22


2021-10-22 20:24:57.501054: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-22 20:24:57.509488: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2601325000 Hz


## <span style="color:blue">Estructura del conjunto de datos</span> 

Un conjunto de datos produce una secuencia de elementos , donde cada elemento tiene la misma estructura (anidada) de componentes . 

Los componentes individuales de la estructura pueden ser de cualquier tipo representable por *tf.TypeSpec*, incluidos *tf.Tensor* , *tf.sparse.SparseTensor* ,*tf.RaggedTensor* , *tf.TensorArray* o *tf.data.Dataset*.

Las construcciones de Python que se pueden usar para expresar la estructura (anidada) de elementos incluyen *tuple , dict , NamedTuple y OrderedDict*. 

En particular, *list* no es una construcción válida para expresar la estructura de los elementos del conjunto de datos. 

Si desea que una entrada de *list* se trate como una estructura, debe convertirla en tuple y si desea que una lista de salida, entonces debe empaquetarla explícitamente usando *tf.stack*.


La propiedad *Dataset.element_spec* permite inspeccionar el tipo de cada componente del elemento. La propiedad devuelve una estructura anidada de objetos *tf.TypeSpec*, que coincide con la estructura del elemento, que puede ser un solo componente, una tupla de componentes o una tupla anidada de componentes. Por ejemplo:

In [7]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))
print(dataset1)

<TensorSliceDataset shapes: (10,), types: tf.float32>


In [8]:
for i in dataset1:
    print(i.numpy())

[0.6204 0.968  0.7465 0.7744 0.0607 0.6962 0.9613 0.2486 0.2937 0.2226]
[0.0055 0.8556 0.8988 0.7702 0.1104 0.528  0.9498 0.9686 0.6508 0.7378]
[0.666  0.9003 0.1938 0.4402 0.2969 0.8741 0.9344 0.2061 0.6413 0.2789]
[0.6481 0.0825 0.2581 0.863  0.6081 0.891  0.0352 0.647  0.4875 0.5228]


In [9]:
len(dataset1)

4

In [10]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]), #y
     tf.random.uniform([4,100], maxval=100, dtype=tf.int32))) #x

dataset2.element_spec
    

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(100,), dtype=tf.int32, name=None))

In [11]:
len(dataset2)

4

In [12]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [13]:
len(dataset3)

4

In [14]:
type(dataset3)

tensorflow.python.data.ops.dataset_ops.ZipDataset

In [15]:
i = iter(dataset3)

In [16]:
print(i.next(), "\n")
print(i.next(), "\n")
print(i.next(), "\n")
print(i.next(), "\n")

(<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.6204, 0.968 , 0.7465, 0.7744, 0.0607, 0.6962, 0.9613, 0.2486,
       0.2937, 0.2226], dtype=float32)>, (<tf.Tensor: shape=(), dtype=float32, numpy=0.8985994>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([97, 50, 15, 40, 46, 86, 62, 13, 41, 58, 54, 65,  7, 94, 22, 78, 93,
       43, 69,  4, 73, 22, 65, 62, 86, 52, 16, 87, 95, 66, 49,  0, 76, 45,
       33, 42, 92, 56, 51, 77, 88, 90,  7, 52, 81, 26, 13, 13, 15, 51, 58,
       59, 11, 47, 32, 73, 88,  6, 87, 47, 61, 22, 22, 90, 47, 92, 21, 85,
       76, 73, 71, 35, 18, 52, 39, 52, 18, 10,  3, 85, 20, 19, 16, 21, 80,
       56, 25, 95, 13, 75, 23,  0, 77, 81, 44, 49,  3, 55, 64, 17],
      dtype=int32)>)) 

(<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.0055, 0.8556, 0.8988, 0.7702, 0.1104, 0.528 , 0.9498, 0.9686,
       0.6508, 0.7378], dtype=float32)>, (<tf.Tensor: shape=(), dtype=float32, numpy=0.32415867>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([1

In [17]:
for a, (b,c) in dataset3:
    print('shapes: {a.shape}, {b.shape}, {c.shape}'.format(a=a, b=b, c=c))

shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)


In [18]:
# dataset con tensores dispersos
dataset4 = tf.data.Dataset.from_tensors(tf.SparseTensor(indices=[[0, 0],[1, 2]], values=[1, 2], dense_shape=[3, 4]))
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [19]:
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

## <span style="color:blue">Leer datos de entrada</span> 

### Consumir matrices Numpy

Si todos sus datos de entrada caben en la memoria, la forma más sencilla de crear un Dataset a partir de ellos es convertirlos en objetos tf.Tensor y usar Dataset.from_tensor_slices() .

In [20]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [21]:
imagenes, labels  = train
imagenes = imagenes /255.

dataset = tf.data.Dataset.from_tensor_slices((imagenes, labels))
dataset

2021-10-22 20:25:02.463024: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 376320000 exceeds 10% of free system memory.


<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

### Consumir generadores de Python

In [22]:
def count(stop):
    i=0
    while i<stop:
        yield i
        i+= 1
        
for n in count(5):
    print(n)

0
1
2
3
4


El constructor `Dataset.from_generator` convierte el generador de Python en un `tf.data.Dataset` completamente funcional.

El constructor toma un invocable como entrada, no un iterador. Esto le permite reiniciar el generador cuando llega al final. Toma un argumento args opcional, que se pasa como argumentos del invocable.

El argumento *output_types* es necesario porque *tf.data* crea un *tf.Graph* internamente y los bordes del gráfico requieren un tf.dtype .




In [23]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes=(),)

In [24]:
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


El argumento `output_shapes` no es necesario, pero se recomienda, ya que muchas operaciones de flujo tensorial no admiten tensores con rango desconocido. Si la longitud de un eje en particular es desconocida o variable, output_shapes puede colcarse como None.

También es importante tener en cuenta que `output_shapes` y `output_types` siguen las mismas reglas de anidamiento que otros métodos de conjuntos de datos.

Aquí hay un generador de ejemplo que demuestra ambos aspectos, devuelve tuplas de matrices, donde la segunda matriz es un vector con longitud desconocida.

In [25]:
def gen_series():
    i = 0
    while True:
        size = np.random.randint(0,10)
        yield i, np.random.normal(size = (size,))
        i+=1

In [26]:
for i, series in gen_series():
    print(i, ":", str(series))
    if i > 5:
        break

0 : [-0.4878 -1.0768  0.1019 -2.1496 -0.1024 -0.3561 -0.5842]
1 : [-1.6571  1.7648]
2 : [ 0.2236 -0.9558 -1.0754]
3 : [ 1.2209 -0.2764 -0.4315  1.0234  1.57    0.6807 -0.1986 -0.6546]
4 : [-0.7463  1.6724  0.5388 -0.1088 -0.2901 -0.6404 -0.1316  0.31  ]
5 : [ 0.5233  0.0156 -0.4458  0.2986  0.2649 -0.009  -0.6764  0.9104  0.7872]
6 : [ 0.1752 -0.7187  0.0144]


La primera salida es un *int32* la segunda es un *float32*.

El primer elemento es un escalar, forma () , y el segundo es un vector de longitud desconocida, forma (None,)


In [27]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types=(tf.int32, tf.float32),
    output_shapes=((), (None, )))

ds_series

<FlatMapDataset shapes: ((), (None,)), types: (tf.int32, tf.float32)>

Ahora se puede utilizar como un *tf.data.Dataset* normal. Tenga en cuenta que al procesar por lotes un conjunto de datos con una forma variable, debe usar *Dataset.padded_batch*.

In [28]:
ds_series_batch = ds_series.shuffle(20).padded_batch(10)

ids, sequence_batch = next(iter(ds_series_batch))

print (ids.numpy())
print()
print(sequence_batch.numpy())

[14 17 20 21 23 15 16 25  4 24]

[[ 0.999   0.0996 -2.2703 -0.8717  0.0303 -1.5378 -2.2933 -2.2715  0.8909]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.1955  0.095   1.036   0.4378  0.694  -1.1221 -0.43    0.      0.    ]
 [-0.7131  1.4196  0.2035 -1.4516  0.2716  0.7313  0.395   0.      0.    ]
 [-1.2149  1.0299 -2.3611 -0.1821  0.3814  0.2352 -0.6536 -2.4488  0.1501]
 [-1.0646  2.0647  0.      0.      0.      0.      0.      0.      0.    ]
 [-0.107   0.8555 -0.8319  0.5519 -2.8758 -0.4358 -0.3965  0.      0.    ]
 [ 0.2342  0.1476  0.2805  0.      0.      0.      0.      0.      0.    ]
 [-1.2992  0.1146  0.355   0.      0.      0.      0.      0.      0.    ]]


In [29]:
it = iter(ds_series_batch)
for i in range(10):
    ids, sequence_batch = next(it)
    print (ids.numpy())
    print()
    print(sequence_batch.numpy())
    print()
    

[10 20  1  9 14 16 15 19 18 28]

[[-2.9010e+00 -2.0369e+00  9.9899e-01  5.8083e-01 -4.7284e-01  1.2962e-01
  -1.2801e+00 -8.1442e-01  5.3743e-02]
 [-1.0548e+00  1.1036e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00]
 [ 1.1315e+00  3.8801e-02  2.1285e+00  1.5725e-03  6.1771e-01 -1.3438e+00
  -7.0964e-02  6.5054e-01 -1.1254e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00]
 [-8.7647e-01 -4.6213e-01  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00]
 [ 1.2491e+00  2.8543e-01  1.7239e+00 -1.7181e-01 -5.0816e-01 -1.3537e+00
   1.1628e-01  2.3577e-01 -1.6698e-01]
 [ 2.1827e-01 -3.0780e-01  3.9623e-01  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00]
 [ 4.6858e-01  4.4233e-01 -1.0819e-01 -1.4063e+00  1.6744e-01 -5.5851e-01
   0.0000e+00  0.0000e+00  0.0000e+00]
 [-3.8315e-01 -4.8301e-01 -1.0096e+00 -9.2611e-01  1.0052e+00  

### Ejemplo realista con imágenes

Para obtener un ejemplo más realista, intente `tf.data.Dataset` `preprocessing.image.ImageDataGenerator` como un `tf.data.Dataset` .

Primero descargue los datos:


In [69]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    cache_dir='/media/storage', #dirección de extracción
    cache_subdir='Datasets', #carpeta que se crea para la extracción
    untar=True)

228818944/228813984 [==============================] - 21s 0us/step


In [70]:
print(flowers)

/media/storage/Datasets/flower_photos


Cree la `image.ImageDataGenerator`

In [71]:
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [72]:
images, labels = next(image_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [ ]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

In [ ]:
ds = tf.data.Dataset.from_generator(
    lambda: image_gen.flow_from_directory(flowers),
    output_types=(tf.float32, tf.float32),
    output_shapes=([32,256,256,3],[32,5]))

ds.element_spec

### Consumir datos de texto

Muchos conjuntos de datos se distribuyen como uno o más archivos de texto. `tf.data.TextLineDataset` proporciona una manera fácil de extraer líneas de uno o más archivos de texto. 

Dados uno o más nombres de archivo, un `TextLineDataset` producirá un elemento con valor de cadena por línea de esos archivos.

In [ ]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url +file_name)
    for file_name in file_names
]

In [ ]:
file_paths

In [ ]:
dataset = tf.data.TextLineDataset(file_paths)

Estas son las primeras líneas del primer archivo:

In [ ]:
for line in dataset.take(5):
    print(line.numpy())

Para alternar líneas entre archivos, use `Dataset.interleave` . Esto facilita la reproducción aleatoria de archivos. Aquí están la primera, segunda y tercera líneas de cada traducción:

In [ ]:
file_ds = tf.data.Dataset.from_tensor_slices(file_paths)

In [ ]:
for i in file_ds: print(i.numpy())

In [ ]:
line_ds = file_ds.interleave(tf.data.TextLineDataset, cycle_length=3)

for i, line in enumerate(line_ds.take(9)):
    if i%3 ==0:
        print()
    print(line.numpy())

De manera predeterminada, `TextLineDataset` produce todas las lineas de cada archivo, lo cual tal vez no sea lo que se quiera. Tal vez el archivo empieza con el encabezado, o contiene comentarios. Para remover o pasarse estas lineas se usan las transformaciones `Dataset.skip()` o `Dataset.filter()`


A continuación, trabajamos con el archivo de la tragedia del Titanic. Se salta la primera linea, y filtramos para tener solo a los sobrevivientes

In [ ]:
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_lines = tf.data.TextLineDataset(titanic_file)

In [ ]:
for line in titanic_lines.take(10):
    print(line.numpy())

In [ ]:
def survived(line):
    return tf.not_equal(tf.strings.substr(line,0,1), '0')

survivors=titanic_lines.skip(1).filter(survived)

In [ ]:
for line in survivors.take(10):
    print(line.numpy())

### Consumir Datos CSV

El formato CSV es muy popular para guardar datos tabulares en forma de texto.

Ya subimos el archivo del titanic, el cual es csv. Podemos subirlo en este mismo formato usando pandas 

In [ ]:
df=pd.read_csv(titanic_file)
df.head()

Si se tiene suficiente memoria, pueden transformar a diccionario el Dataframe e importar los datos con facilidad

In [ ]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

Un acercamiento más ameno es cargar desde el disco cuando sea necesario.

el modulo tiene métodos para extraer rgistros de uno o más archivos CSV que cumplan con la [RFC 4180](https://tools.ietf.org/html/rfc4180)

la función `experimental.make_csv_dataset` es una interfaz para leer conjuntos de archivos CSV, con lo cual podemos hacer inferencia por columna y crear lotes de los datos

Se puede usar el argumento `select_columns` si solo se necesitan algunas columnas

In [ ]:
titanic_batches = tf.data.experimental.make_csv_dataset(
    titanic_file, batch_size=4,
    label_name="survived", select_columns=['class', 'fare', 'survived'])

In [ ]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived': {}".format(label_batch))
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

### Consumir conjuntos de archivos

Es normal que los datos estén distribuidos en múltiples archivos, con cada archivo teniendo ejemplos

In [ ]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
flowers_root = pathlib.Path(flowers_root)

Cada directorio de la carpeta raíz contiene un directorio de cada clase

In [ ]:
for item in flowers_root.glob("*"):
  print(item.name)

Cada archivo en los directorios son ejemplos

In [ ]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

for f in list_ds.take(5):
  print(f.numpy())

usando `tf.io.read_file` podemos ler los datos y extraer las etiquetas, obteniendo (imagen, etiqueta)

In [ ]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

labeled_ds = list_ds.map(process_path)

In [ ]:
for image_raw, label_text in labeled_ds.take(1):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())

## Loteo de elementos del dataset

### Loteo simple

La transformación `Dataset.batch()` es la forma más sencilla de hacer un lote de `n` elementos consecutivos. Para cada componente, todos los elementos deben tener un tensor de exactamente la misma dimensión

In [ ]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

for batch in batched_dataset.take(4):
  print([arr.numpy() for arr in batch])

### Loteo de tensores con acolchamiento

Con el loteo simple todos los tensores debe tener la misma dimensión, pero esto no va a ser el caso todas las veces. Utilizando `Dataset.padded_batch` se hace un acolchamiento de los tensores de distintas formas, específicando las dimensiones a las cuales hay que aplicar acolchamiento

In [ ]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
dataset = dataset.padded_batch(4, padded_shapes=(None,))

for batch in dataset.take(2):
  print(batch.numpy())
  print()

## Flujo de entrenamiento

### Procesando múltiples epochs

La API ofrece dos maneras dde procesar múltiples epochs de los mismos datos.

La primera manera es iterando sobre el el conjunto de datos utilizando `Dataset.repeat()`. volvemos al ejemplo de texto del Titanic. 

In [ ]:
def plot_batch_sizes(ds):
  batch_sizes = [batch.shape[0] for batch in ds]
  plt.bar(range(len(batch_sizes)), batch_sizes)
  plt.xlabel('Batch number')
  plt.ylabel('Batch size')

`Dataset.repeat` hace una concatenación de los argumentos sin señalar el inicio o el final de un epoch. Si aplicamos
`Dataset.batch` Después de esta, se producirán lotes que van más allá de los límites e los epochs.

Si la función `repeat` no tiene argumentos, se hara una repetición infinita.

In [ ]:
titanic_batches = titanic_lines.repeat(3).batch(128)
plot_batch_sizes(titanic_batches)

si queremos una separación clara de los epoch, se aplica `Dataset.batch` antes de `repeat`

In [ ]:
titanic_batches = titanic_lines.batch(128).repeat(3)

plot_batch_sizes(titanic_batches)

Si queremos, por ejemplo, recopilar estadísticas al final de cada epoch, podemos hacer una iteración y reiniciar después de cada epoch

In [ ]:
epochs = 3
dataset = titanic_lines.batch(128)

for epoch in range(epochs):
  for batch in dataset:
    print(batch.shape)
  print("End of epoch: ", epoch)

## Mezclar los datos de entrada

la transformación `Dataser.shuffle()` toma una muestra de un tamaño predeterminado y selecciona el siguiente dato del buffer.

Le agregaremos un indice a los datos del titanic para que el efecto sea visible

In [ ]:
lines = tf.data.TextLineDataset(titanic_file)
counter = tf.data.experimental.Counter()

dataset = tf.data.Dataset.zip((counter, lines))
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(20)
dataset

In [ ]:
n,line_batch = next(iter(dataset))
print(n.numpy())

`shuffle` no señala el fin de un epoch hasta que el buffer esté vacío. si aplicamos `repeat` antes de este, se podrá ver el momento en el que termina un epoch y empieza otro

In [ ]:
dataset = tf.data.Dataset.zip((counter, lines))
shuffled = dataset.shuffle(buffer_size=100).batch(10).repeat(2)

print("esta es la lista de indices cercanos al fin del epoch:\n")
for n, line_batch in shuffled.skip(60).take(5):
  print(n.numpy())

gráficamente se puede apreciar mejor

In [ ]:
shuffle_repeat = [n.numpy().mean() for n, line_batch in shuffled]
plt.plot(shuffle_repeat, label="shuffle().repeat()")
plt.ylabel("Mean item ID")
plt.legend()

si ponemos `repeat`antes de la mezcla, los límites de los epoch se mantendrán iguales hasta que no hayan más objetos que mezclar

In [ ]:
dataset = tf.data.Dataset.zip((counter, lines))
shuffled = dataset.repeat(2).shuffle(buffer_size=100).batch(10)

print("esta es la lista de indices cercanos al fin del epoch:\n")
for n, line_batch in shuffled.skip(55).take(15):
  print(n.numpy())

In [ ]:
repeat_shuffle = [n.numpy().mean() for n, line_batch in shuffled]

plt.plot(shuffle_repeat, label="shuffle().repeat()")
plt.plot(repeat_shuffle, label="repeat().shuffle()")
plt.ylabel("Mean item ID")
plt.legend()

## Preprocesamiento de datos

si se quiere aplicar alguna función a los datos en cuestión, se utiliza  la transformación `Dataset.map(f)`. Esta toma los objetos `t f.Tensor` de un solo elemento y saca nuevos objetos en un nuevo conjunto de datos.

Aquí mostramos dos ejemplos muy comunes de pre procesamiento

### Decodificando imagenes y cambiar su tamaño

Al trabajar con imagenes de la vida cotidiana, lo más probable es que necesitemos estandarizar los tamaños a uno en común. 

Utilizaremos la lista de flores para este ejemplo

In [ ]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

escribimos una función para manipular datos

In [ ]:
# Lee una imagen de un archivo, la decodifica en un tensor y cambia su tamaño
# a una forma predeterminada
def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]

  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [128, 128])
  return image, label

In [ ]:
file_path = next(iter(list_ds))
image, label = parse_image(file_path)

def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label.numpy().decode('utf-8'))
  plt.axis('off')

show(image, label)

In [ ]:
images_ds = list_ds.map(parse_image)

for image, label in images_ds.take(2):
  show(image, label)

### Aplicando funciones de python

Por razones de rendimiento, es mejor usar únicamente funciones de Tensorflow para manipular datos, pero a veces es necesario usar herramientas de otros paquetes de python.

Para esto utilizamos `tf.py_function()` como función en `Dataset.map()`

Supongamos que queremos hacer una rotación aleatoria en un conjunto dde imágenes. Tensorflow sólo tiene `tf.image.rot90`, lo cual no sirve para la intención que se tiene. por suerte, el paquete scipy cuenta con `scipy.ndimage.rotate`

In [ ]:
import scipy.ndimage as ndimage

def random_rotate_image(image):
  image = ndimage.rotate(image, np.random.uniform(-30, 30), reshape=False)
  return image

In [ ]:
image, label = next(iter(images_ds))
image = random_rotate_image(image)
show(image, label)

In [ ]:
def tf_random_rotate_image(image, label):
  im_shape = image.shape
  [image,] = tf.py_function(random_rotate_image, [image], [tf.float32])
  image.set_shape(im_shape)
  return image, label

In [ ]:
rot_ds = images_ds.map(tf_random_rotate_image)

for image, label in rot_ds.take(2):
  show(image, label)

## Ventaneo De series de tiempo

En el caso de modelos de series de tiempo, estos datos están organizados con el axis de tiempo intacto. Muchas veces se le alimentaran secciones de tiempo adyacentes a los modelos como datos. Hay dos maneras de generar estos cortes. La primera es utilizando lotes:

In [ ]:
range_ds = tf.data.Dataset.range(100000)

In [ ]:
batches = range_ds.batch(10, drop_remainder=True)

for batch in batches.take(5):
  print(batch.numpy())

Para hacer predicciones un paso hacia el futuro, es ideal mover los datos y etiquetas un paso relativo a ellos

In [ ]:
def dense_1_step(batch):
  # Se mueven las características y etiquetas un paso hacia la derecha
  return batch[:-1], batch[1:]

predict_dense_1_step = batches.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
  print(features.numpy(), " => ", label.numpy())

Para predecir una ventana completa de tiempo, podemos separar los lotes en dos partes

In [ ]:
batches = range_ds.batch(15, drop_remainder=True)

def label_next_5_steps(batch):
  return (batch[:-5],   # Se toman los primeros 10 pasos
          batch[-5:])   # se toma el residuo

predict_5_steps = batches.map(label_next_5_steps)

for features, label in predict_5_steps.take(3):
  print(features.numpy(), " => ", label.numpy())

Para permitir que se superpongan las características de un lote y las etiquetas de otro, podemos usar `Dataset.zip()`

In [ ]:
feature_length = 10
label_length = 3

features = range_ds.batch(feature_length, drop_remainder=True)
labels = range_ds.batch(feature_length).skip(1).map(lambda labels: labels[:label_length])

predicted_steps = tf.data.Dataset.zip((features, labels))

for features, label in predicted_steps.take(5):
  print(features.numpy(), " => ", label.numpy())

Por supuesto, a veces se necesitan más control de las ventanas. Razón por la que se puede usar `Dataset.window`, pero para usarla correctamente, necesitamos algo de cuidado en lso datos. Esta transformación retorna un conjunto de conjuntos de datos

In [ ]:
window_size = 5

windows = range_ds.window(window_size, shift=1)
for sub_ds in windows.take(5):
  print(sub_ds)

¿Pero qué pasó aquí? para ver los datos como un solo conjunto, usamos `Dataset.flat_map`. Al mismo tiempo casi siempre es necesario hacer lotes

In [ ]:
for x in windows.flat_map(lambda x: x).take(30):
   print(x.numpy(), end=' ')

In [ ]:
def sub_to_batch(sub):
  return sub.batch(window_size, drop_remainder=True)

for example in windows.flat_map(sub_to_batch).take(5):
  print(example.numpy())

Haciéndolo todo junto, obtendríamos una función como esta

In [ ]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
  windows = ds.window(window_size, shift=shift, stride=stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

  windows = windows.flat_map(sub_to_batch)
  return windows

In [ ]:
ds = make_window_dataset(range_ds, window_size=10, shift = 5, stride=3)

for example in ds.take(10):
  print(example.numpy())

Es sencillo extraer etiquetas con estos datos

In [ ]:
dense_labels_ds = ds.map(dense_1_step)

for inputs,labels in dense_labels_ds.take(3):
  print(inputs.numpy(), "=>", labels.numpy())

##  Remuestreo

Es usual encontrarse con datasets desbalanceados a nivel de clases. Es buena idea aquí el hacer un remuestreo del dataset. `tf.data` da dos métodos para esto

Se usará el dataset de fraude de tarjetas de crédito es perfecto para demostrarlo

In [74]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/download.tensorflow.org/data/creditcard.zip',
    fname='creditcard.zip',
    cache_dir='/media/storage',
    cache_subdir='Datasets',
    extract=True)

csv_path = zip_path.replace('.zip', '.csv')

69156864/69155632 [==============================] - 6s 0us/step


In [75]:
creditcard_ds = tf.data.experimental.make_csv_dataset(
    csv_path, batch_size=1024, label_name="Class",
    # Set the column types: 30 floats and an int.
    column_defaults=[float()]*30+[int()])

Se revisará ahora la distribución de las clases a clasificar, para ver qué tan sesgados están

In [ ]:
def count(counts, batch):
  features, labels = batch
  class_1 = labels == 1
  class_1 = tf.cast(class_1, tf.int32)

  class_0 = labels == 0
  class_0 = tf.cast(class_0, tf.int32)

  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)

  return counts

In [ ]:
counts = creditcard_ds.take(10).reduce(
    initial_state={'class_0': 0, 'class_1': 0},
    reduce_func = count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy()]).astype(np.float32)

fractions = counts/counts.sum()
print(fractions)

Para poder trabajar con datos desbalanceados, la mejor idea es balancearlos. Aquí algunos métodos para esto

### Muestreo de Datasets

La forma más sencilla es usar `sample_from_datasets`. Esto es particularmente mejor cuando se tienen datasets separados por clase. Para este caso se va a filtrar los datos de fraude para esta razón

In [ ]:
negative_ds = (
  creditcard_ds
    .unbatch()
    .filter(lambda features, label: label==0)
    .repeat())
positive_ds = (
  creditcard_ds
    .unbatch()
    .filter(lambda features, label: label==1)
    .repeat())

In [ ]:
for features, label in positive_ds.batch(10).take(1):
  print(label.numpy())

Se pasarán los datasets, junto con los pesos que se quieren por `tf.data.experimental.sample_from_datasets`

In [ ]:
balanced_ds = tf.data.experimental.sample_from_datasets(
    [negative_ds, positive_ds], [0.5, 0.5]).batch(10)

Ahora se generarán ejemplos de las clases con una probabilidad 50/50

In [ ]:
for features, labels in balanced_ds.take(10):
  print(labels.numpy())

## Remuestreo de rechazo

Como se dijo, necesitamos que los datasets estén separados por clase. Podemos por supuesto usar `Dataset.filter`, pero eso haría que los datos se cargaran dos veces.

La función `data.experimental.rejection_resample` permite rebalancear los datos sin tener que cargarlos otra vez. Esto se logra eliminando elementos del dataset para llegar al balance.

Esta función toma un argumento `class_func`. Esta función es aplicada a cada elemento del dataset para determinar la clase que tiene.

Los elementos de `creditcard_ds` ya están separados en pares `(features,label)`. Así que la función solo tiene que retornar la etiqueta

In [ ]:
def class_func(features, label):
  return label

De igual forma es necesaria una distribución objetivo y preferiblemente un estimado de esta

In [ ]:
resampler = tf.data.experimental.rejection_resample(
    class_func, target_dist=[0.5, 0.5], initial_dist=fractions)

`resampler` trabaja con las observaciones de manera individual, así que hay que aplicar `unbatch()` antes.

In [ ]:
resample_ds = creditcard_ds.unbatch().apply(resampler).batch(10)

el resampler retorna pares del tipo `(class, example)` a partír de la salida de `class_func`. En este caso ya tenemos `(feature, label)`, así que se hace un map para obtener una copia extra de las etiquetas

In [ ]:
balanced_ds = resample_ds.map(lambda extra_label, features_and_label: features_and_label)

In [ ]:
for features, labels in balanced_ds.take(10):
  print(labels.numpy())

Cuál es el problema de este método? si el desbalanceo es muy grande, se va a perder una cantidad muy grande de datos. ¿Qué es más importante: La cantidad de datos o la cantidad de recursos?